## Proyecto Final
### Hugo Villanueva
#### Ciencia de Datos en Python
#### Postgrado en análisis y predicción de datos

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl

In [5]:
main_data = np.load('proyecto_training_data.npy')

In [6]:
len(main_data)

1460

In [14]:
training_data = main_data[0:1168] # El 80% de la cantidad de datos es 1168
test_data = main_data[1168:] #El resto de los datos

In [15]:
print(np.shape(test_data))
print(np.ndim(test_data))

(292, 6)
2
